In [1]:
from awrams.simulation.ondemand import OnDemandSimulator
from awrams.models import awral

from awrams.utils import extents
from awrams.utils import datetools as dt
from awrams.utils.precision import quantize

import pandas as pd
import os
os.makedirs('./_results',exist_ok=True)

In [2]:
def change_path_to_forcing(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join,dirname
    from os import getcwd

    data_path = '../../test_data/simulation/'

    FORCING = {
        'tmin': ('temp_min*','temp_min_day',data_path),
        'tmax': ('temp_max*','temp_max_day',data_path),
        'precip': ('rain_day*','rain_day',data_path),
        'solar': ('solar*','solar_exposure_day',data_path)
    }
    for k,v in FORCING.items():
        imap.mapping[k+'_f'] = nodes.forcing_from_ncfiles(v[2],v[0],v[1])


In [3]:
def get_initial_states_dict(imap,period,extent):
    from awrams.utils.io.data_mapping import SplitFileManager
    from awrams.utils.nodegraph import nodes

    from os.path import join,dirname
    from os import getcwd

    data_path = '../../test_data/simulation/'
    
    node_names = {'mleaf_dr': 'init_mleaf_hrudr',
                  'mleaf_sr': 'init_mleaf_hrusr',
                  's0_dr': 'init_s0_hrudr',
                  's0_sr': 'init_s0_hrusr',
                  'ss_dr': 'init_ss_hrudr',
                  'ss_sr': 'init_ss_hrusr',
                  'sd_dr': 'init_sd_hrudr',
                  'sd_sr': 'init_sd_hrusr',
                  'sg_bal': 'init_sg',
                  'sr_bal': 'init_sr'}

    data_map = {}
    period = [period[0] - 1]
    
    for k in node_names:
        sfm = SplitFileManager.open_existing(data_path,k+'*nc',k)
        data_map[node_names[k]] = sfm.get_data(period,extent)

    nodes.init_states_from_dict(imap,data_map,extent)


In [4]:
def insert_climatology(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join,dirname
    from os import getcwd

    data_path = '../../test_data/simulation/'

    imap.mapping['precip_f'] = nodes.forcing_gap_filler(data_path,'rain_day*','rain_day',join(data_path,'climatology_daily_rain_day.nc'))
    imap.mapping['tmax_f'] = nodes.forcing_gap_filler(data_path,'temp_max*','temp_max_day',join(data_path,'climatology_daily_temp_max_day.nc'))
    imap.mapping['tmin_f'] = nodes.forcing_gap_filler(data_path,'temp_min*','temp_min_day',join(data_path,'climatology_daily_temp_min_day.nc'))
    imap.mapping['solar_f'] = nodes.forcing_gap_filler(data_path,'solar*','solar_exposure_day',join(data_path,'climatology_daily_solar_exposure_day.nc'))


In [5]:
sites = {
'A1' : (148.106488,-35.497492),
'A3' : (148.101076,-35.399688),
'A5' : (148.085427,-35.360194),
# 'K1' : (147.55912 ,-35.49322),
# 'K2' : (147.53052 ,-35.43525),
# 'K3' : (147.56893 ,-35.43408),
# 'K4' : (147.6     ,-35.42688),
# 'K5' : (147.60408 ,-35.41928),
# 'K6' : (147.4572  ,-35.38978),
# 'K10': (147.5348  ,-35.32395),
# 'K11': (147.42902 ,-35.27202),
# 'K13': (147.5333  ,-35.23887),
# 'M1' : (148.970567,-36.293033),
# 'M2' : (149.2     ,-35.3088),
# 'M4' : (147.196183,-33.938267),
# 'M7' : (146.07    ,-34.249),
# 'Y1' : (145.84895 ,-34.62888),
# 'Y4' : (146.02003 ,-34.71943),
# 'Y5' : (146.29317 ,-34.72835),
# 'Y7' : (146.1153  ,-34.85183),
# 'Y8' : (146.41398 ,-34.84697),
# 'Y9' : (146.01632 ,-34.96777),
# 'Y10': (146.30988 ,-35.00535),
# 'Y11': (145.93553 ,-35.10975),
# 'Y12': (146.16893 ,-35.0696),
# 'Y13': (146.30648 ,-35.09025),
}

for k in sorted(sites.keys()):
    print(k,quantize(sites[k][1],0.05),quantize(sites[k][0],0.05))

A1 -35.5 148.1
A3 -35.4 148.1
A5 -35.35 148.1


In [6]:
period = dt.dates('dec 2010 - jan 2011')

In [7]:
grid_names = ['slope_grid',
 'kssat_v5_grid',
 'ne_grid',
 'pref_grid',
 'kdsat_v5_grid',
 'hveg_dr_grid',
 'lai_max_grid',
 's0fracawc_grid',
 'ssfracawc_grid',
 'f_tree_grid',
 'k0sat_v5_grid',
 'meanpet_grid',
 'k_gw_grid']

sm_cols = ('s0','s0_sr','s0_dr','ss','ss_sr','ss_dr','sd','sd_sr','sd_dr',)

forcing = ('tmin','tmax','solar_f','precip_f')

In [8]:
grids = {}

for site in sorted(sites):
    print()
    extent = extents.from_cell_coords(quantize(sites[site][1],0.05),quantize(sites[site][0],0.05))
    print(site,extent)
    
    imap = awral.get_default_mapping()

#     change_path_to_forcing(imap)
    insert_climatology(imap)
    get_initial_states_dict(imap,period,extent)
    
    runner = OnDemandSimulator(awral,imap.mapping) #,omapping=omap.mapping)
    r,i = runner.run(period,extent,return_inputs=True)
    
    sm = {k:r[k].reshape(-1) for k in sm_cols}
    df = pd.DataFrame(sm,index=period)
    df.to_csv('./_results/'+site+'_soil_moisture.csv', float_format='%.4f')

    clm = {k:i[k].reshape(-1) for k in forcing}
    df = pd.DataFrame(clm,index=period)
    df.to_csv('./_results/'+site+'_forcing.csv', float_format='%.2f')
    
    grids[site] = {k:i[k][0] for k in grid_names}

    
df = pd.DataFrame(grids).T
df.to_csv('./_results/grid_input_parameters.csv') #, float_format='%.4f')


A1 lat: -35.5 (510) lon: 148.1 (722) shape:(1, 1)

A3 lat: -35.4 (508) lon: 148.1 (722) shape:(1, 1)

A5 lat: -35.35 (507) lon: 148.1 (722) shape:(1, 1)
